In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter import PhotoImage
from tkinter import ttk
import requests
from PIL import Image, ImageTk
from dotenv import load_dotenv
import os
import datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score


load_dotenv()
api_key = os.getenv("api_key")

if not api_key:
    print("API key not found! Please check your .env file.")
    exit(1)

trained_model = None

def get_current_weather():
    city = city_entry.get()
    if not city:
        messagebox.showerror("Input Error", "Please enter a city name.")
        return

    result_label.config(text="Loading current weather...")
    base_url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}"

    try:
        response = requests.get(base_url)
        data = response.json()

        if response.status_code == 200:
            print(f"Data fetched for city: {city}")  
            location = data["location"]["name"]
            country = data["location"]["country"]
            temperature = data["current"]["temp_c"]
            condition = data["current"]["condition"]["text"]
            condition_icon = data["current"]["condition"]["icon"]
            humidity = data["current"]["humidity"]
            wind_speed = data["current"]["wind_kph"]

           
            icon_url = f"http:{condition_icon}"
            img = Image.open(requests.get(icon_url, stream=True).raw)
            img = img.resize((50, 50))
            img = ImageTk.PhotoImage(img)

            result_label.config(
                text=f"Current Weather in {location}, {country}\n"
                     f"Temperature: {temperature}°C\n"
                     f"Condition: {condition}\n"
                     f"Humidity: {humidity}%\n"
                     f"Wind Speed: {wind_speed} km/h"
            )
            icon_label.config(image=img)
            icon_label.image = img
        else:
            messagebox.showerror("Error", f"Error fetching data: {data['error']['message']}")
    except Exception as e:
        messagebox.showerror("API Error", f"An error occurred: {str(e)}")
        result_label.config(text="")


def get_historical_weather():
    city = city_entry.get()
    start_date = start_date_entry.get()
    end_date = end_date_entry.get()

    if not city:
        messagebox.showerror("Input Error", "Please enter a city name.")
        return
    if not start_date or not end_date:
        messagebox.showerror("Input Error", "Please enter a valid date range.")
        return

    try:
        start_date_obj = datetime.datetime.strptime(start_date, "%Y-%m-%d")
        end_date_obj = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    except ValueError:
        messagebox.showerror("Input Error", "Dates must be in YYYY-MM-DD format.")
        return

    result_label.config(text="Loading historical data...")
    base_url = f"http://api.weatherapi.com/v1/history.json?key={api_key}&q={city}"

    all_data = []
    current_date = start_date_obj
    while current_date <= end_date_obj:
        date_str = current_date.strftime("%Y-%m-%d")
        try:
            print(f"Fetching data for {date_str}") 
            response = requests.get(f"{base_url}&dt={date_str}")
            data = response.json()

            if "forecast" in data:
                for day in data["forecast"]["forecastday"]:
                    record = {
                        "date": day["date"],
                        "temperature": day["day"]["avgtemp_c"],
                        "humidity": day["day"]["avghumidity"],
                        "wind_speed": day["day"]["maxwind_kph"],
                        "condition": day["day"]["condition"]["text"]
                    }
                    all_data.append(record)

            current_date += datetime.timedelta(days=1)
        except Exception as e:
            messagebox.showerror("API Error", f"An error occurred: {str(e)}")
            return

   
    if all_data:
        df = pd.DataFrame(all_data)
        result_label.config(text=f"Historical Weather Data for {city}:\n{df.to_string(index=False)}")
        train_weather_model(df)
    else:
        result_label.config(text="No historical data found.")

# the weather prediction model
def train_weather_model(df):
    global trained_model

    try:
        print("Training model...") 
        df["condition"] = df["condition"].astype("category").cat.codes

        X = df[["temperature", "humidity", "wind_speed"]]
        y = df["condition"]

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

      
        model = GaussianNB()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        trained_model = model

        messagebox.showinfo("Model Training", f"Model trained successfully with accuracy: {accuracy:.2f}")
    except Exception as e:
        messagebox.showerror("Model Training Error", f"An error occurred during training: {str(e)}")

def predict_weather():
    global trained_model

    if trained_model is None:
        messagebox.showerror("Prediction Error", "Model is not trained yet. Fetch historical data first.")
        return

    try:
        temperature = float(temp_entry.get())
        humidity = float(humidity_entry.get())
        wind_speed = float(wind_speed_entry.get())

        sample_data = pd.DataFrame({
            "temperature": [temperature],
            "humidity": [humidity],
            "wind_speed": [wind_speed]
        })

        prediction = trained_model.predict(sample_data)
        result_label.config(text=f"Predicted Weather Condition Code: {prediction[0]}")
    except Exception as e:
        messagebox.showerror("Prediction Error", f"An error occurred: {str(e)}")


root = tk.Tk()
root.title("Weather App with Prediction")
root.geometry("600x700")
root.config(bg="#f1f1f1")

frame = tk.Frame(root, bg="#6a94d2", relief="flat")
frame.pack(pady=10, padx=20, fill="both")

tk.Label(frame, text="Enter City:", font=("Arial", 14), bg="#6a94d2", fg="#fff").grid(row=0, column=0, padx=10)
city_entry = tk.Entry(frame, font=("Arial", 14))
city_entry.grid(row=0, column=1, padx=10)

tk.Label(frame, text="Start Date (YYYY-MM-DD):", font=("Arial", 12), bg="#6a94d2", fg="#fff").grid(row=1, column=0, padx=10)
start_date_entry = tk.Entry(frame, font=("Arial", 12))
start_date_entry.grid(row=1, column=1, padx=10)

tk.Label(frame, text="End Date (YYYY-MM-DD):", font=("Arial", 12), bg="#6a94d2", fg="#fff").grid(row=2, column=0, padx=10)
end_date_entry = tk.Entry(frame, font=("Arial", 12))
end_date_entry.grid(row=2, column=1, padx=10)

tk.Button(frame, text="Get Current Weather", command=get_current_weather, bg="#f57c00", fg="#fff", font=("Arial", 12)).grid(row=3, column=0, pady=10)
tk.Button(frame, text="Get Historical Weather", command=get_historical_weather, bg="#f57c00", fg="#fff", font=("Arial", 12)).grid(row=3, column=1, pady=10)

tk.Label(root, text="Predict Weather:", font=("Arial", 14), bg="#f1f1f1").pack(pady=10)
prediction_frame = tk.Frame(root, bg="#e3e3e3", relief="flat")
prediction_frame.pack(pady=10, padx=20, fill="both")

tk.Label(prediction_frame, text="Temperature (°C):", font=("Arial", 12)).grid(row=0, column=0, padx=10)
temp_entry = tk.Entry(prediction_frame, font=("Arial", 12))
temp_entry.grid(row=0, column=1, padx=10)

tk.Label(prediction_frame, text="Humidity (%):", font=("Arial", 12)).grid(row=1, column=0, padx=10)
humidity_entry = tk.Entry(prediction_frame, font=("Arial", 12))
humidity_entry.grid(row=1, column=1, padx=10)

tk.Label(prediction_frame, text="Wind Speed (km/h):", font=("Arial", 12)).grid(row=2, column=0, padx=10)
wind_speed_entry = tk.Entry(prediction_frame, font=("Arial", 12))
wind_speed_entry.grid(row=2, column=1, padx=10)

tk.Button(prediction_frame, text="Predict Weather", command=predict_weather, bg="#6a94d2", fg="#fff", font=("Arial", 12)).grid(row=3, column=0, columnspan=2, pady=10)

icon_label = tk.Label(root, bg="#f1f1f1")
icon_label.pack()

result_label = tk.Label(root, text="", font=("Arial", 12), bg="#f1f1f1", wraplength=550)
result_label.pack(pady=20)

root.mainloop()


API key not found! Please check your .env file.
